In [9]:
import numpy as np
import cv2

In [10]:
image = cv2.imread("Images/2_colour.jpeg")
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png")

In [11]:
def ExtractObject(S2, ObjectMask):
    final_im = mask*image
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [33]:
def ObjectBlender(s1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene s1"""
    alpha = 0.5
    beta = 1 - alpha
    blendedObject = cv2.addWeighted(s1, alpha, FilteredExObject, beta, 0.0)
    cv2.imshow('Blended', blendedObject)
    cv2.waitKey(0)
    return blendedObject


In [34]:
extracted = ExtractObject(image, mask)

In [35]:
img = cv2.imread("Images/1_colour.jpeg", 1)
blended = ObjectBlender(img, extracted)